In [14]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain-openai


In [15]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")   


In [16]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict): #creating state as an extension of dict
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4o-2024-08-06',
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)



For the code bellow, we interact with the LLM using the graph; the user input is entered at the **entrypoint** and then the graph (Agents) are gonna process the output of other agents

In [18]:
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [19]:
_set_env("TAVILY_API_KEY")

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool.invoke("hiring values morgan stanley") #could use to search company names and classify them

[{'url': 'https://www.morganstanley.com/people-opportunities/students-graduates/culture',
  'content': 'Core Values. Everything we do at Morgan Stanley is guided by our five core values: Do the right thing, put clients first, lead with exceptional ideas, commit to diversity and inclusion, and give back. Since our founding in 1935, Morgan Stanley has consistently delivered first-class business in a first-class way.'},
 {'url': 'https://www.indeed.com/cmp/Morgan-Stanley/about',
  'content': 'Since our founding in 1935, underpinning our history and legacy are five core values. By putting clients first, leading with exceptional ideas, doing the right thing, committing to diversity and inclusion, and giving back, Morgan Stanley aims to deliver tangible results for today with visionary thinking for the future.'}]

In [21]:
llm_with_tools = llm.bind_tools(tools)#bind toolkit to node

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)